In [1]:
import torch
import numpy as np

from models.TS.GRUPipeline import GRUwithFNO
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import boto3

In [2]:
SYMBOL = "BTCUSDT"

session = boto3.session.Session()
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='YCAJEEvXFkR_Vlz_q0TNbK1f7',
    aws_secret_access_key='YCPw07kPNEgOJVF0N93yReOErAl7RP0-5woP_Bgl',
    region_name='ru-central1'
)

bucket_name = 'test-actions'

x_train_file = f'train/train_x_{SYMBOL}.pt'
y_train_file = f'train/train_y_{SYMBOL}.pt'
x_test_file = f'test/test_x_{SYMBOL}.pt'
y_test_file = f'test/test_y_{SYMBOL}.pt'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

s3.download_file(bucket_name, x_train_file, x_train_file[6:])
s3.download_file(bucket_name, y_train_file, y_train_file[6:])
s3.download_file(bucket_name, x_test_file, x_test_file[5:])
s3.download_file(bucket_name, y_test_file, y_test_file[5:])

print(x_train_file[6:])
x_train, y_train = torch.load(f'train_x_{SYMBOL}.pt', weights_only=True).to(torch.float32).to(device), torch.load(
    f'train_y_{SYMBOL}.pt', weights_only=True).to(torch.float32).to(device)
x_test, y_test = torch.load(f'test_x_{SYMBOL}.pt', weights_only=True).to(torch.float32).to(device), torch.load(
    f'test_y_{SYMBOL}.pt', weights_only=True).to(torch.float32).to(device)

mean, std = x_train.squeeze().reshape(-1).detach().mean().item(), x_train.squeeze().reshape(
    -1).detach().std().item()
dataset = TensorDataset(x_train, y_train)
validation = TensorDataset(x_test, y_test)

train_x_BTCUSDT.pt


In [3]:
gru = GRUwithFNO(input_size=20, sequence_length=10, hidden_size=30, num_layers=2, output_length=10, components=1, mean=mean, scale=std)
print(f'model/weights_{SYMBOL}', f'weights_{SYMBOL}')
s3.download_file(bucket_name, f'model/weights_{SYMBOL}.pt', f'weights_{SYMBOL}.pt')
gru.load_state_dict(torch.load(f'weights_{SYMBOL}.pt', weights_only=False, map_location=torch.device(device)))

model/weights_BTCUSDT weights_BTCUSDT


<All keys matched successfully>

In [4]:
gru.eval()
print(x_test[0][np.newaxis, :, :].shape)
x = x_test[0][np.newaxis, :, :].detach()
print(x_test)
pred = gru(x)
print(pred)

torch.Size([1, 10, 20])
tensor([[[71079.8906, 71075.9219, 71091.0781,  ..., 70928.8906,
          70977.5391, 70996.3594],
         [71075.9219, 71091.0781, 71125.1484,  ..., 70977.5391,
          70996.3594, 71066.8438],
         [71091.0781, 71125.1484, 71091.1016,  ..., 70996.3594,
          71066.8438, 71049.1094],
         ...,
         [71052.0625, 71004.5391, 71022.3125,  ..., 71004.0391,
          71047.3516, 71049.5000],
         [71004.5391, 71022.3125, 71029.6562,  ..., 71047.3516,
          71049.5000, 71054.5781],
         [71022.3125, 71029.6562, 71015.4219,  ..., 71049.5000,
          71054.5781, 71055.3906]],

        [[71038.3125, 71015.6016, 71010.7578,  ..., 70828.1328,
          70847.1484, 70833.2109],
         [71015.6016, 71010.7578, 71015.8516,  ..., 70847.1484,
          70833.2109, 70851.6484],
         [71010.7578, 71015.8516, 70965.9375,  ..., 70833.2109,
          70851.6484, 70881.9766],
         ...,
         [71024.8594, 71044.4531, 71060.8906,  ..., 708